In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


<img src='../../../common/logo_DH.png' align='left' width=35%/>

# Preprocesamiento en Text Mining.

En esta práctiva vamos a usar el dataset de mails que ya usamos en la clase de Naive Bayes.

Este dataset (<a href="https://www.kaggle.com/riyadhrazzaq/multinomial-naive-bayes-from-scratch/data?select=spam.csv" traget="_blank">fuente</a>) tiene dos columnas:

* una columna con el cuerpo del mail, 

* y otra con la etiqueta spam / ham según corresponda a un mail que es spam o no respectivamente.

Vamos a 
* preprocesar los textos de los mails usando `CountVectorizer` y `TfidfTransformer`
* usar Singular Value Decomposition para obtener una representación en dos dimensiones de cada mail, 
* y contruir nubes de palabras para visualizar que palabras caracterizan a cada una de la etiquetas (spam / ham).


## Imports

In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt

Si el import de `WordCloud` da error, pueden instalarla descomentando esta linea:

In [ ]:
#! conda install -c conda-forge wordcloud=1.8.1 --yes --name dhdsblend2021

## Ejercicio 1

Leer los datos del archivo `spam.csv`. 

Mantener sólo las columnas 1 y 2 etiquetarlas como 'target' y 'text'

Ayuda: usar `encoding='iso8859_14'`

In [ ]:
data = pd.read_csv("../Data/spam.csv", encoding='iso8859_14')
data.head()

In [ ]:
data.drop(labels=data.columns[2:],axis=1,inplace=True)
data.columns=['target','text']
data.head()

## Ejercicio 2

Los textos del dataset están en idioma inglés.

Construir una lista de stems de stopwords usando `SnowballStemmer`

Generar una instancia de DataFrame resultado de aplicar CountVectorizer a los textos de los mails, usando como stopwords la lista resultado del paso anterior.

A partir del encoding resultado de CountVectorizer, generar una instancia de DataFrame resultado de aplicar TfidfTransformer.

In [ ]:
stopwords_en = stopwords.words('english');

englishStemmer = SnowballStemmer("english")

# si no hacemos esto y usamos directo stopwords_sp, CountVectorizer devuelve un warning
stopwords_en_stem = [englishStemmer.stem(x) for x in stopwords_en]

vectorizer = CountVectorizer(stop_words = stopwords_en_stem, lowercase = True, strip_accents = 'unicode');

vectorizer.fit(data.text);

#print('Vocabulario:\n',vectorizer.vocabulary_) # vocabulario del corpus con la frecuencia de cada término


In [ ]:
countvectorizer_encoding = vectorizer.transform(data.text);

pd.DataFrame(countvectorizer_encoding.todense(), 
             columns = vectorizer.get_feature_names()) # Usamos el método .todense() para ver la matriz completa

In [ ]:
tfidf_encoding = TfidfTransformer().fit_transform(countvectorizer_encoding);

pd.DataFrame(tfidf_encoding.todense(),columns = vectorizer.get_feature_names())


## Ejercicio 3

Representar en un scatterplot las dos componenetes más importantes que resultan de Singular Value Decomposition de la representación tf-idf, coloreando los puntos de acuerdo a su etiqueta.

In [ ]:
svd = TruncatedSVD(n_components=2);

data_components = svd.fit_transform(tfidf_encoding)

data_components_df = pd.DataFrame(data_components)

In [ ]:
sns.scatterplot(x = data_components_df.iloc[:, 0], y = data_components_df.iloc[:, 1], hue = data.target)

## Ejercicio 4

Representar en un scatterplot las dos componenetes más importantes que resultan de Singular Value Decomposition de la representación CountVectorizer, coloreando los puntos de acuerdo a su etiqueta.

¿Cuál de las dos (ej 3 vs ej 4) les resulta más informativa?

¿Creen que algún algoritmo de clustering agruparía adecuadamente los tipos de email? ¿Por qué?

In [ ]:
svd = TruncatedSVD(n_components=2);

data_components_cv = svd.fit_transform(countvectorizer_encoding)

data_components_cv_df = pd.DataFrame(data_components_cv)

In [ ]:
sns.scatterplot(x = data_components_cv_df.iloc[:, 0], y = data_components_cv_df.iloc[:, 1], hue = data.target)

## Ejercicio 5

Usando este <a href="https://www.python-graph-gallery.com/wordcloud/" target="_blank">tutorial</a> o este  <a href="https://towardsdatascience.com/simple-wordcloud-in-python-2ae54a9f58e5" target="_blank">otro</a> generen las nubes de palabras para 

* los registros de etiqueta spam
* los registros de etiqueta ham

¿El resultado concuerda con la intuición que tienen sobre los textos de mails spam?

In [ ]:
mask_spam = data.target == 'spam'
data_spam = data.loc[mask_spam, :]
text_spam = ' '.join(data_spam.text)

In [ ]:
# Create the wordcloud object
wordcloud = WordCloud(width=1500, height=1500, margin=0,random_state=1, stopwords = stopwords_en).generate(text_spam)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

In [ ]:
mask_ham = np.logical_not(mask_spam)
data_ham = data.loc[mask_ham, :]
text_ham = ' '.join(data_ham.text)

In [ ]:
# Create the wordcloud object
wordcloud = WordCloud(width=1500, height=1500, margin=0, random_state=1,stopwords = stopwords_en).generate(text_ham)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

In [ ]:
# Define a function to plot word cloud
def plot_cloud(wordcloud):
    # Set figure size
    plt.figure(figsize=(30, 20))
    # Display image
    plt.imshow(wordcloud) 
    # No axis details
    plt.axis("off");

In [ ]:
wordcloud_spam = WordCloud(width = 2000, height = 1500, 
                      random_state=1, background_color='steelblue', 
                      colormap='Pastel1', collocations=False, 
                      stopwords = stopwords_en).generate(text_spam)

plot_cloud(wordcloud_spam)

In [ ]:
wordcloud_ham = WordCloud(width = 2000, height = 1500, 
                      random_state=1, background_color='steelblue', 
                      colormap='Pastel1', collocations=False, 
                      stopwords = stopwords_en).generate(text_ham)

plot_cloud(wordcloud_ham)